In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
def write_file(json_data, nome_arquivo):
    """
    Escreve dados em um arquivo JSON, adicionando ao arquivo existente se ele já existir.

    Parâmetros:
    - json_data (list): Lista de dados em formato JSON a serem escritos no arquivo.
    - nome_arquivo (str): Nome do arquivo onde os dados serão escritos ou adicionados.

    Exemplo de uso:
    ```python
    json_list = [{'order_id': 1, 'product': 'Item 1'}, {'order_id': 2, 'product': 'Item 2'}]
    write_file(json_list, 'orders.json')
    ```

    Se o arquivo já existir, os dados fornecidos serão adicionados aos dados existentes.
    Se o arquivo não existir, um novo arquivo será criado e os dados serão escritos nele.
    """
    if os.path.exists(nome_arquivo):
        with open(nome_arquivo, "r") as arquivo_existente:
            dados_existente = json.load(arquivo_existente)

        dados_existente.extend(json_data)

        with open(nome_arquivo, "w") as arquivo:
            json.dump(dados_existente, arquivo)
    else:
        with open(nome_arquivo, "w") as arquivo:
            json.dump(json_data, arquivo)

#### Verificando dados de usuário

In [3]:
#### Verificando dados de usuário
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4544,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1563,"completed":34558,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

In [4]:
# # Selecionar data da pesquisa
# # date_from = "2023-11-1"
# # date_to = "2023-11-3"

# # date_from = "2023-11-3"
# # date_to = "2023-11-5"

# # date_from = "2023-11-5"
# # date_to = "2023-11-7"

# # date_from = "2023-11-8"
# # date_to = "2023-11-10"

# # date_from = "2023-11-10"
# # date_to = "2023-11-12"

# date_from = "2023-11-12"
# date_to = "2023-11-14"

# date_from = "2023-11-14"
# date_to = "2023-11-16"


# date_from = "2023-11-16"
# date_to = "2023-11-18"

# date_from = "2023-11-18"
# date_to = "2023-11-20"

# date_from = "2023-11-20"
# date_to = "2023-11-22"

# date_from = "2023-11-22"
# date_to = "2023-11-24"

# date_from = "2023-11-24"
# date_to = "2023-11-26"

# date_from = "2023-11-26"
# date_to = "2023-11-28"

# date_from = "2023-11-28"
# date_to = "2023-11-30"

# date_from = "2023-11-30"
# date_to = "2023-12-2"

# # date_from = "2023-12-2"
# # date_to = "2023-12-4"

# # date_from = "2023-12-4"
# # date_to = "2023-12-6"

# # URL base da API
# base_url = "https://api.mercadolibre.com/orders/search"

# # Parâmetros iniciais
# params = {
#     "seller": "233632476",
#     "order.date_closed.from": f"{date_from}T00:00:00.000-03:00",
#     "order.date_closed.to": f"{date_to}T00:00:00.000-03:00",
#     "limit": 50,
#     "offset": 0,
# }

# headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

# json_list = []

# counter = 0

# try:
#     while True:
#         response = requests.get(base_url, params=params, headers=headers)
#         response.raise_for_status()  # Lança uma exceção se a resposta não for bem-sucedida
#         data = response.json()

#         if "results" in data:
#             json_list.extend(data["results"])
#         else:
#             break

#         # Verifique se há mais páginas
#         if "paging" in data:
#             total_paging = data["paging"].get("total")
#             if total_paging is None:
#                 break

#             total_pages = math.ceil(total_paging / params["limit"])
#             print(f"Total esperado de páginas: {counter + 1}/{total_pages}")
#             print(f"Total de dados esperados: {total_paging}")
#             print(f'Offset atual: {params["offset"]}')

#             counter += 1
#             # if params["offset"] >= total_paging:
#             if params["offset"] > total_paging:
#                 break

#             params["offset"] += params["limit"]
#         else:
#             break

# except requests.exceptions.RequestException as req_err:
#     print(f"Erro ao fazer a requisição para {base_url}: {req_err}")

# except Exception as e:
#     print(f"Erro não esperado: {e}")

# print(f"Total de dados coletados: {len(json_list)}")

In [7]:
json_list = []

In [8]:
# Função para obter dados para um intervalo de datas específico
def get_data_for_date_range(date_from, date_to, offset=0, limit=50):
    base_url = "https://api.mercadolibre.com/orders/search"

    params = {
        "seller": "233632476",
        "order.date_closed.from": f"{date_from}T00:00:00.000-03:00",
        "order.date_closed.to": f"{date_to}T00:00:00.000-03:00",
        "limit": limit,
        "offset": offset,
    }

    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

    json_list = []
    counter = 0

    try:
        while True:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()

            if "results" in data:
                json_list.extend(data["results"])
            else:
                break

            if "paging" in data:
                total_paging = data["paging"].get("total")
                if total_paging is None:
                    break

                total_pages = math.ceil(total_paging / params["limit"])
                print(f"Total esperado de páginas: {counter + 1}/{total_pages}")
                print(f"Total de dados esperados: {total_paging}")
                print(f'Offset atual: {params["offset"]}')
                print(f"Intervalo de datas atual: {date_from} - {date_to}")

                if params["offset"] > total_paging:
                    break

                params["offset"] += params["limit"]
                counter += 1
            else:
                break

    except requests.exceptions.RequestException as req_err:
        print(f"Erro ao fazer a requisição para {base_url}: {req_err}")

    except Exception as e:
        print(f"Erro não esperado: {e}")

    print(f"Total de dados coletados para {date_from} - {date_to}: {len(json_list)}")
    return json_list


# Definir os intervalos de datas desejados
date_ranges = [
    ("2023-11-1", "2023-11-3"),
    ("2023-11-2", "2023-11-5"),
    ("2023-11-4", "2023-11-7"),
    ("2023-11-6", "2023-11-9"),
    ("2023-11-8", "2023-11-11"),
    ("2023-12-10", "2023-11-13"),
    ("2023-11-12", "2023-11-15"),
    ("2023-11-14", "2023-11-17"),
    ("2023-11-16", "2023-11-19"),
    ("2023-11-18", "2023-11-21"),
    ("2023-11-20", "2023-11-23"),
    ("2023-11-22", "2023-11-25"),
    ("2023-11-24", "2023-11-27"),
    ("2023-11-26", "2023-11-29"),
    ("2023-11-28", "2023-12-1"),
    ("2023-11-30", "2023-12-3"),
    ("2023-12-2", "2023-12-5"),
    ("2023-12-4", "2023-12-7"),
    ("2023-12-6", "2023-12-8"),
    # Adicione mais intervalos conforme necessário
]

# Executar a função para cada intervalo de datas
for date_range in date_ranges:
    date_from, date_to = date_range
    json_data = get_data_for_date_range(date_from, date_to)
    json_list.extend(json_data)

# Agora, 'json_list' contém os dados para todos os intervalos de datas
print(f"Total de dados coletados para todos os intervalos: {len(json_list)}")

Total esperado de páginas: 1/5
Total de dados esperados: 227
Offset atual: 0
Intervalo de datas atual: 2023-11-1 - 2023-11-3
Total esperado de páginas: 2/5
Total de dados esperados: 227
Offset atual: 50
Intervalo de datas atual: 2023-11-1 - 2023-11-3
Total esperado de páginas: 3/5
Total de dados esperados: 227
Offset atual: 100
Intervalo de datas atual: 2023-11-1 - 2023-11-3
Total esperado de páginas: 4/5
Total de dados esperados: 227
Offset atual: 150
Intervalo de datas atual: 2023-11-1 - 2023-11-3
Total esperado de páginas: 5/5
Total de dados esperados: 227
Offset atual: 200
Intervalo de datas atual: 2023-11-1 - 2023-11-3
Total esperado de páginas: 6/5
Total de dados esperados: 227
Offset atual: 250
Intervalo de datas atual: 2023-11-1 - 2023-11-3
Total de dados coletados para 2023-11-1 - 2023-11-3: 227
Total esperado de páginas: 1/9
Total de dados esperados: 419
Offset atual: 0
Intervalo de datas atual: 2023-11-2 - 2023-11-5
Total esperado de páginas: 2/9
Total de dados esperados: 41

In [ ]:
print(len(json_list))
# json_list

In [ ]:
write_file(json_list, "../../Data/Output/orders.json")

In [ ]:
# with open("../../Data/Output/orders.json", "r") as file:
# # with open("orders_05_06.json", "r") as file:
#     json_list = json.load(file)

# len(json_list)

### Coletando dados de orders


In [ ]:
resultados = []

for item in json_list:
    # Extrair os valores desejados
    payments = item.get("payments", [])
    status = item["status"]
    date_closed = item["date_closed"]
    pack_id = item["pack_id"]
    shipping = item["shipping"]
    order_items = item.get("order_items", [])
    fulfilled = item["fulfilled"]

    for payment in payments:
        order_id = payment["order_id"]
        reason = payment["reason"]
        payment_status = payment["status"]
        date_approved = payment["date_approved"]

    # Inicializa variável para armazenar shipping_id
    shipping_id = None
    shipping_id = shipping["id"]

    # # Inicializa listas para armazenar informações específicas de order_items

    # Itera sobre os dicionários em order_items
    for order_item in order_items:
        item_info = order_item.get("item", {})
        # item_id = item_info.get("id")
        ml_code = item_info["id"]
        title = item_info["title"]
        variation_id = item_info["variation_id"]
        seller_sku = item_info["seller_sku"]
        quantity = order_item["quantity"]
        category_id = item_info["category_id"]

        variation_attributes = item_info.get("variation_attributes", [])

        name = None
        value_id = None
        value_name = None
        for attribute in variation_attributes:
            name = attribute["name"]
            id = attribute["id"]
            value_id = attribute["value_id"]
            value_name = attribute["value_name"]

    # Adicionar os resultados à lista
    resultados.append(
        {
            # "payments": payments,
            "ml_code": ml_code,
            "payment_status": payment_status,
            "order_status": status,
            "order_id": order_id,
            "shipping_id": shipping_id,
            "pack_id": pack_id,
            "title": title,
            "variation_id": variation_id,
            "category_id": category_id,
            "seller_sku": seller_sku,
            "quantity": quantity,
            "variation_name": name,
            "variation_attributes_id": id,
            "variation_value_id": value_id,
            "variation_value_name": value_name,
            "date_approved": date_approved,
            "date_closed": date_closed,
        }
    )

# Exibir os resultados
pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(resultados)

# Tratando dados numéricos
pd.set_option("display.float_format", "{:.0f}".format)
df["shipping_id"] = df["shipping_id"].fillna(0)
df["pack_id"] = df["pack_id"].fillna(0)
df["variation_id"] = df["variation_id"].fillna(0)
df["shipping_id"] = df["shipping_id"].astype("int64")
df["pack_id"] = df["pack_id"].astype("int64")
df["variation_id"] = df["variation_id"].astype("int64")


# Adiciona 1h a mais para chegar ao horário do Brasil
df["date_approved"] = pd.to_datetime(df["date_approved"])
df["date_closed"] = pd.to_datetime(df["date_closed"])
df["date_approved"] = df["date_approved"] + pd.to_timedelta("1 hour")
df["date_approved"] = df["date_approved"].dt.tz_localize(None)
df["date_closed"] = df["date_closed"] + pd.to_timedelta("1 hour")
df["date_closed"] = df["date_closed"].dt.tz_localize(None)


print(df.shape)
df.sample(10)
# df.head(4)

In [ ]:
# df_nan = df[df['shipping_id'].isna()]

# df_nan = df[df.isnull().any(axis=1)]

# # Exibir o DataFrame resultante
# df_nan

In [ ]:
df.dtypes

In [ ]:
x = df[df["shipping_id"] == "4278213349"]
x = df[df["shipping_id"] == 42782133490]
x = df[df["ml_code"] == "MLB4166151108"]
# x = df[df['order_id'] == 2000006836057908]
x

## Coletando logistic type

In [ ]:
import requests

json_logistic_list = []

for shipping_id in df["shipping_id"]:
    url = f"https://api.mercadolibre.com/shipments/{shipping_id}"

    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

        json_data = response.json()
        json_logistic_list.append(json_data)  # Adiciona o JSON da resposta à lista
        print(f"Obtido com sucesso para shipping_id {shipping_id}: {json_data}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter dados para shipping_id {shipping_id}: {e}")

    # time.sleep(5)
# Exibir os resultados
print(json_logistic_list)

In [ ]:
# import requests
# import time

# json_logistic_list = []

# for i, shipping_id in enumerate(df['shipping_id'], start=1):
#     url = f"https://api.mercadolibre.com/shipments/{shipping_id}"
#     print(i,':',url)
#     headers = {
#         'Authorization': f'Bearer {ACCESS_TOKEN}'
#     }

#     try:
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

#         json_data = response.json()
#         json_logistic_list.append(json_data)  # Adiciona o JSON da resposta à lista
#         print(f"Obtido com sucesso para shipping_id {shipping_id}: {json_data}")

#         # Pausa a cada 50 consultas por 1 minuto
#         if i % 50 == 0:
#             print(f"Atingido limite de 50 consultas. Aguardando 1 minuto...")
#             time.sleep(60)

#     except requests.exceptions.RequestException as e:
#         print(f"Erro ao obter dados para shipping_id {shipping_id}: {e}")

# # Exibir os resultados
# print(json_logistic_list)

In [ ]:
print(len(json_logistic_list))
print(len(json_list))

In [ ]:
len(json_logistic_list) == len(json_list)

In [ ]:
write_file(json_logistic_list, "../../Data/Output/shipping.json")

In [ ]:
# with open("../../Data/Output/shipping.json", "r") as file:
# # with open("orders_05_06.json", "r") as file:
#     json_logistic_list = json.load(file)

# len(json_logistic_list)

### Adicionando logistic_type ao df principal

In [ ]:
dfx = pd.DataFrame(json_logistic_list)
# cols = ['id', 'order_id', 'logistic_type']
cols = ["id", "logistic_type"]
dfx = dfx[cols]
# dfx['id'].value_counts()

print(dfx.shape)
dfx.head(3)

In [ ]:
# Criar um dicionário a partir de dfx para mapear 'order_id' para 'logistic_type'
order_id_to_logistic_type = dfx.set_index("id")["logistic_type"].to_dict()

# Adicionar a coluna 'logistic_type' a df usando o mapeamento
df_result = df.copy()  # Criar uma cópia de df para manter o original intacto
df_result["logistic_type"] = df_result["shipping_id"].map(order_id_to_logistic_type)

print(df_result.shape)
df_result.head(3)

In [ ]:
# x = df_result[df_result['shipping_id'] == 42782133490]
# x = df_result[df_result['ml_code'] == 'MLB4166151108']
# x = df_result[df_result['order_id'] == 2000006859129842]
x = df_result[pd.isna(df_result["logistic_type"])]

x

In [ ]:
df_result[["order_id", "shipping_id", "pack_id", "variation_id"]] = df_result[
    ["order_id", "shipping_id", "pack_id", "variation_id"]
].astype(str)
df_result[["order_id", "shipping_id", "pack_id", "variation_id"]]

In [ ]:
df_result.dtypes

## Populando tabela

In [ ]:
df_result["variation_attributes_id"] = df_result["variation_attributes_id"].replace(
    "<built-in function id>", None, inplace=True
)

df_result[df_result["ml_code"] == "MLB1991060554"]
df_result[df_result["order_id"] == "2000006887872016"]
# df[df['ml_code']=='MLB1991060554']

In [ ]:
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in df_result.iterrows():
    # for index, row in df_resultado.iterrows():
    # insert_query = sql.SQL(
    #     "INSERT INTO ml_orders_hist (ml_code,category_id,variation_id,seller_sku,pack_id,quantity,title,order_status,payment_status,shipping_id,date_approved,date_closed,fulfilled) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s)"
    # )
    insert_query = sql.SQL(
        "INSERT INTO ml_orders (ml_code,category_id,variation_id,seller_sku,order_id,pack_id,quantity,title,order_status,payment_status,variation_name,variation_attributes_id,variation_value_id,variation_value_name,shipping_id,date_approved,date_closed,logistic_type) VALUES (%s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)"
    )
    print("Inserindo dados:", [value for value in row])
    # print("Tipos de dados dos valores:", [type(value) for value in row])

    cursor.execute(
        insert_query,
        (
            row["ml_code"],
            row["category_id"],
            row["variation_id"],
            row["seller_sku"],
            row["order_id"],
            row["pack_id"],
            row["quantity"],
            row["title"],
            row["order_status"],
            row["payment_status"],
            row["variation_name"],
            row["variation_attributes_id"],
            row["variation_value_id"],
            row["variation_value_name"],
            row["shipping_id"],
            row["date_approved"],
            row["date_closed"],
            row["logistic_type"],
        ),
    )

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print("Dados inseridos com sucesso!")